## Script to generate FATES parameter files for one at a time ensemble

#### Libraries

In [1]:
from scipy.stats import qmc
import numpy as np
import xarray as xr
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re

import functions_py.modp as mp


#### Set the seed

In [2]:
random.seed(32)

- Read in main csv
- For each parameter:
- if it is global simply create min and max param file
- if it is pft specific read in the relevant csv and make min and max changing the relevant pfts 

In [3]:
param_map = pd.read_csv('/qfs/people/need138/fates-fbnc-cal/param_dir/fbnc_params/FATES_FBNC_calibration_params.csv')
param_map = param_map[['Parameter_Name', 'PFT', 'Min_value', 'Max_value']]

# remove allometry parameters for now - param ranges will come from 
# analysis outside of FATES
tmp = param_map.Parameter_Name.str.contains('allom')
param_map = param_map[tmp == False]

param_map = param_map.reset_index(drop=True)
print(param_map)


# number of parameters
n_params = len(param_map)
print(n_params)
param_names = list(param_map.Parameter_Name)
pfts = list(param_map.PFT)

# param_scalars = [i for i, x in enumerate(list(param_ranges.parameter)) if re.search('pftscalar', x)]
# print(param_scalars)


                         Parameter_Name  PFT  Min_value   Max_value
0      fates_maintresp_nonleaf_baserate    0   0.000001    0.000006
1   fates_maintresp_reduction_intercept    1   0.000000    1.000000
2                          fates_grperc    1   0.100000    1.000000
3           fates_alloc_storage_cushion    1   1.000000    3.000000
4                    fates_wood_density    1        NaN         NaN
5                      fates_mort_bmort    1   0.010000    0.050000
6         fates_mort_scalar_cstarvation    1   0.100000    1.000000
7         fates_mort_scalar_hydrfailure    1   0.100000    1.000000
8               fates_mort_disturb_frac    0   0.000000    1.000000
9          fates_mort_understorey_death    0   0.000000    1.000000
10                fates_turnover_branch    1  75.000000  225.000000
11                  fates_turnover_leaf    1        NaN         NaN
12


#### Make parameter files

In [4]:
## Read in defaut FATES file - note that this is the default for FATES but with:
# - size bins that are consistent with Piponoit et al.
# - updated vai params
# - martinez_cano and chave allometries for trees
# - no max height
print(param_map.PFT)
print(param_map.PFT[10])

input_fname = '/qfs/people/need138/fates-fbnc-cal/param_dir/fates_params_fbnc_base.nc'

# for each parameter
for i in range(0,n_params) :
    print(i)
    
    param = param_names[i]
    print(param)
    pft = param_map.PFT[i]
     
    # final parameter file names
    fout_min = '/qfs/people/need138/fates-fbnc-cal/param_dir/fates_params_fbnc_{0}_min.nc'.format(param)
    fout_max = '/qfs/people/need138/fates-fbnc-cal/param_dir/fates_params_fbnc_{0}_max.nc'.format(param)
    
    shutil.copyfile(input_fname, fout_min)   
    shutil.copyfile(input_fname, fout_max)   
    
    # Global parammeters
    if pft == 0 : 
        val_min = param_map.Min_value[i]
        val_max = param_map.Max_value[i]
            
        mp.main(var = param, pft = 0, fin = fout_min, val = val_min, 
            fout = fout_min, O = 1, organ = 0)
        mp.main(var = param, pft = 0, fin = fout_max, val = val_max, 
            fout = fout_max, O = 1, organ = 0)

    # PFT specific parameters   
    else :
        # load the pft file
        pft_param_map = pd.read_csv('/qfs/people/need138/fates-fbnc-cal/param_dir/fbnc_params/{0}.csv'.format(param))
        pft_param_map = pft_param_map[['Parameter', 'PFT', 'Min', 'Max']]
    
        for p in range(0,len(pft_param_map.PFT)) :
            
            pfts = pft_param_map.PFT
            
            val_min = pft_param_map.Min[p]
            val_max = pft_param_map.Max[p]
               
            mp.main(var = param, pft = pfts[p], fin = fout_min, val = val_min, 
                    fout = fout_min, O = 1, organ = 0)
            mp.main(var = param, pft = pfts[p], fin = fout_max, val = val_max, 
                    fout = fout_max, O = 1, organ = 0)

    

0     0
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     0
9     0
10    1
11    1
Name: PFT, dtype: int64
1
0
fates_maintresp_nonleaf_baserate
1
fates_maintresp_reduction_intercept
2
fates_grperc
3
fates_alloc_storage_cushion
4
fates_wood_density
5
fates_mort_bmort
6
fates_mort_scalar_cstarvation
7
fates_mort_scalar_hydrfailure
8
fates_mort_disturb_frac
9
fates_mort_understorey_death
10
fates_turnover_branch
11
fates_turnover_leaf
